# Lab) Kaggle Competition for Beginner

#### Data Preprocess
- Data Load
- Data Set of Train, Valid, Test
- Feature Engineerning (To Do)
- Data Scaling & Encoding

#### Model Tranding
- Hyper Parameter Tune: Optuna
- Model Traning

#### Ensemble
- Out-of-fold(OOF) Ensemble

#### Submission
- Make submission

In [ ]:
# 나눔 글꼴 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (9,672 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120882 files and direc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from os.path import join
import multiprocessing

import numpy as np
import pandas as pd

# 시각화
import matplotlib.pylab as plt
import matplotlib as mpl
import seaborn as sns
import re

%matplotlib inline
plt.rc('font', family='NanumBarunGothic')

import warnings

warnings.filterwarnings('ignore')

n_cpus = multiprocessing.cpu_count() - 1

##Data Preprocess
### 1. Data Load
- pandas.read_csv()

In [ ]:
ROOT_PATH = '/content/drive/MyDrive/ASAC/kaggle특강/실습/'

In [ ]:
train_path = join(ROOT_PATH, 'data', 'train.csv')
test_path  = join(ROOT_PATH, 'data', 'test.csv')
submission_path = join(ROOT_PATH, 'data', 'sample_submission.csv')

In [ ]:
data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
data.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [ ]:
test.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,74051,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,74052,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,74053,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,74054,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,74055,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


데이터 설명
- id: 각 게 데이터의 고유한 식별자로, 데이터를 구분하는데 사용됩니다.
- Sex: 게의 성별을 나타내는 컬럼으로, 주로 두 가지 값으로 표현됩니다. 예를 들어, "M"은 수컷(male)을, "F"는 암컷(female)을 나타낼 수 있습니다.., "I"는 불확실한 개체
- Length: 게의 길이를 나타내는 컬럼으로, 일반적으로 밀리미터(mm) 단위로 기록됩니다.
- Diameter: 게의 지름을 나타내는 컬럼으로, 마찬가지로 밀리미터(mm) 단위로 기록됩니다.
- Height: 게의 높이를 나타내는 컬럼으로, 역시 밀리미터(mm) 단위로 기록됩니다.
- Weight: 게의 총 무게를 나타내는 컬럼으로, 일반적으로 그램(g) 단위로 표시됩니다.
- Shucked Weight: 게의 껍질을 제거한 후의 무게를 나타내는 컬럼으로, 그램(g) 단위로 기록됩니다.
- Viscera Weight: 게 내장의 무게를 나타내는 컬럼으로, 그램(g) 단위로 표현됩니다.
- Shell Weight: 게의 껍질의 무게를 나타내는 컬럼으로, 그램(g) 단위로 기록됩니다.
- Age(Target): 게의 나이를 나타내는 컬럼으로, 주로 "년"이나 "년수"로 표시되며, 게의 연령을 추정하기 위해 사용됩니다.

In [ ]:
# 라벨 컬럼을 별도의 변수로 빼둡니다.
label = data['Age']

In [ ]:
data.head(20)

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8
5,5,M,1.5000,1.1750,0.4125,28.845616,13.409313,6.789705,7.937860,10
6,6,M,1.5750,1.1375,0.3500,30.022120,11.935140,7.342521,8.646598,11
7,7,I,1.3125,1.0250,0.3500,18.299602,8.249704,3.898056,5.669900,11
8,8,F,1.6000,1.2875,0.4375,38.824640,16.967176,7.413394,10.772810,12
9,9,M,1.0250,0.7625,0.2625,10.305043,4.493396,2.126212,2.976698,11


In [ ]:
data.describe()#include='all')

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000
mean,37025.000000,1.317460,1.024496,0.348089,23.385217,10.104270,5.058386,6.723870,9.967806
std,21376.826729,0.287757,0.237396,0.092034,12.648153,5.618025,2.792729,3.584372,3.175189
min,0.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.042524,0.042524,1.000000
25%,18512.500000,1.150000,0.887500,0.300000,13.437663,5.712424,2.863300,3.968930,8.000000
50%,37025.000000,1.375000,1.075000,0.362500,23.799405,9.908150,4.989512,6.931453,10.000000
75%,55537.500000,1.537500,1.200000,0.412500,32.162508,14.033003,6.988152,9.071840,11.000000
max,74050.000000,2.012815,1.612500,2.825000,80.101512,42.184056,21.545620,28.491248,29.000000


In [ ]:
#
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74051 entries, 0 to 74050
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              74051 non-null  int64  
 1   Sex             74051 non-null  object 
 2   Length          74051 non-null  float64
 3   Diameter        74051 non-null  float64
 4   Height          74051 non-null  float64
 5   Weight          74051 non-null  float64
 6   Shucked Weight  74051 non-null  float64
 7   Viscera Weight  74051 non-null  float64
 8   Shell Weight    74051 non-null  float64
 9   Age             74051 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 5.6+ MB


In [ ]:
test.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,74051,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,74052,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,74053,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,74054,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,74055,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


In [ ]:
test.describe()

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
count,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000
mean,98734.500000,1.319795,1.026357,0.348693,23.464627,10.130220,5.072615,6.745876
std,14251.458382,0.286950,0.236593,0.092259,12.641703,5.611779,2.791563,3.575650
min,74051.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.014175,0.042524
25%,86392.750000,1.162500,0.887500,0.300000,13.536886,5.740774,2.877474,3.968930
50%,98734.500000,1.387500,1.075000,0.362500,23.813580,9.979024,4.989512,6.945627
75%,111076.250000,1.537500,1.200000,0.412500,32.290081,14.047177,7.016501,9.171063
max,123418.000000,2.037500,1.625000,2.825000,80.101512,42.184056,21.545620,28.491248


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              49368 non-null  int64  
 1   Sex             49368 non-null  object 
 2   Length          49368 non-null  float64
 3   Diameter        49368 non-null  float64
 4   Height          49368 non-null  float64
 5   Weight          49368 non-null  float64
 6   Shucked Weight  49368 non-null  float64
 7   Viscera Weight  49368 non-null  float64
 8   Shell Weight    49368 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 3.4+ MB


### Feature Engineering(To Do)

In [ ]:
# Feature Engineering
# To do

#### 전처리

##### 불필요한 컬럼 삭제
- id 컬럼은 식별자의 역할을 할 뿐 유의미한 feature로 만들기 어렵습니다. 제거하도록 합니다.

In [ ]:
data.drop(columns=['id', 'Age'], inplace=True)

test.drop(columns=['id'], inplace=True)

##### 수치형 변수 컬럼 및 범주형 변수 컬럼

In [ ]:
# DF.info에서 Object type을 가진 컬럼은 모두 범주형 변수
cat_columns = data.select_dtypes(include='object').columns
num_columns = data.select_dtypes(exclude='object').columns

print('Categorical Columns: \n{}\n\n Numeric Columns: \n{}\n'.format(cat_columns, num_columns))

Categorical Columns: 
Index(['Sex'], dtype='object')

 Numeric Columns: 
Index(['Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight'],
      dtype='object')



In [ ]:
data.reset_index()

,index,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395
...,...,...,...,...,...,...,...,...,...
74046,74046,F,1.6625,1.2625,0.4375,50.660556,20.680960,10.361742,12.332033
74047,74047,I,1.0750,0.8625,0.2750,10.446791,4.323299,2.296310,3.543687
74048,74048,F,1.4875,1.2000,0.4125,29.483480,12.303683,7.540967,8.079607
74049,74049,I,1.2125,0.9625,0.3125,16.768729,8.972617,2.919999,4.280774


#### 전처리 프로세스 함수로 만들기 (실습 15분)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# 결측치 처리 방식 : iterativeImputer
# MICE 알고리즘 https://www.google.com/search?q=mice%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98&sca_esv=582466614&rlz=1C1CHBD_koKR1068KR1068&tbm=isch&sxsrf=AM9HkKkONyGsaVONyjKl9Ir2vWqztXVI4g:1700011219828&source=lnms&sa=X&ved=2ahUKEwjvw_ym68SCAxVvslYBHT9jC4QQ_AUoAXoECAIQAw&biw=1116&bih=785&dpr=1.13#imgrc=-47AlBaN8pHeBM
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()

    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    tmp_x_test.reset_index(drop=True, inplace=True)

    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
    ohe.fit(tmp_x_train[cat_columns])
    ohe_columns = ohe.get_feature_names_out()

    new_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]), columns=ohe_columns)
    new_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]), columns=ohe_columns)
    new_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]), columns=ohe_columns)

    tmp_x_train = pd.concat([tmp_x_train[num_columns], new_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid[num_columns], new_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test[num_columns],  new_test_cat], axis=1)

    return tmp_x_train, tmp_x_valid, tmp_x_test

## Cross Validation
### 1. Train, Valid, Test Set
훈련, 검증, 테스트 데이터라고 부르는 3가지를 한번 이야기 해보겠습니다.<br>
* Train Data : 모델을 학습하는데 사용하는 데이터 (모델이 알고 있는 학습할 데이터, 과거 데이터)
* Valid Data : 학습한 모델의 성능을 검증하는 데이터 (모델이 모르는 학습하지 않을 데이터, 모델 검증에 사용하는 데이터, 과거 데이터)
* Test Data : 학습한 모델로 예측할 데이터 (모델이 모르는 예측할 데이터, 미래 데이터)
* 데이터 유출이 있지는 않았는지 등 검증이 필요하다.

<img src='https://t1.daumcdn.net/cfile/tistory/9951E5445AAE1BE025' style='height : 500px' >


##### 데이터 쪼개기, Train -> (Train, Valid)
- train_test_split 파라미터
    - test_size  (float): Valid(test)의 크기의 비율을 지정
    - random_state (int): 데이터를 쪼갤 때 내부적으로 사용되는 난수 값 (해당 값을 지정하지 않으면 매번 달라집니다.)
    - shuffle     (bool): 데이터를 쪼갤 때 섞을지 유무
    - stratify   (array): Stratify란, 쪼개기 이전의 클래스 비율을 쪼개고 나서도 유지하기 위해 설정해야하는 값입니다. 클래스 라벨을 넣어주면 됩니다.
        - ex) 원본 Train 데이터의 클래스 비율이 (7:3) 이었다면, 쪼개어진 Train, Valid(test) 데이터의 클래스 비율도 (7:3)이 됩니다. 당연히 분류 데이터에서만 사용할 수 있습니다.

#### ref
- [Validation 데이터가 필요한 이유](https://3months.tistory.com/118)

In [ ]:
np.linspace(0,10,11)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [ ]:
stratify_label = np.digitize(label, np.linspace(0,10,11))

In [ ]:
from sklearn.model_selection import train_test_split

# 쪼개어진 Train, Valid 데이터의 비율은 (7:3), 내부 난수 값 42, 데이터를 쪼갤 때 섞으며 label 값으로 Stratify 하는 코드 입니다. random_state를 주석 처리하고 데이터를 확인해보시면 계속 바뀝니다.
x_train, x_valid, y_train, y_valid = train_test_split(data, label,
                                                      test_size=0.3,
                                                      random_state=42,
                                                      shuffle=True,
                                                      stratify=stratify_label)

In [ ]:
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((51835, 8), (22216, 8), (51835,), (22216,))

## AutoML
### Optuna
데이터의 형태 혹은 도메인에 따라 성능이 좋거나, 괜찮다고 알려진 모델들은 많습니다. 하지만 해당 모델들의 기본 설정 값을 이용한다면 우리 데이터에 좋은 성능을 낼 수 있을까요? 대부분 그렇지 않습니다. 따라서 우리 데이터에 좋은 성능을 낼 수 있도록 모델 파라미터를 조정해주어야합니다. 이를 Hyter Parameter Tuning, HPO 라 합니다.

많이 알려진 라이브러리로 Optuna, HyperOpt, Scikit-Opt 등이 있지만, 우리 수업에서는 조금 더 직관적이고 사용하기 쉬운 Optuna에 대해 다룹니다.


#### 주요 특징
- 가볍고 다양한 플랫폼에 구애 받지 않는 구조
- 조건 및 루프를 통한 손쉬운 Search Space 정의
- 효율적인 최적화 알고리즘
- 쉬운 병렬화
- 빠른 시각화

#### ref
- [Optuna, Documentation](https://optuna.readthedocs.io/en/stable/index.html)


- Optuna

storage: (기본값: None)

의미: Study의 상태를 저장할 스토리지를 지정합니다.
가능한 선택지: 문자열 또는 storages.BaseStorage 객체

sampler: (기본값: None)

의미: 실험 구성을 어떻게 선택할지를 결정하는 샘플러를 지정합니다.
가능한 선택지: 문자열 또는 samplers.BaseSampler 객체

pruner: (기본값: None)

의미: 조기 중단을 결정하는 프루너를 지정합니다.
가능한 선택지: 문자열 또는 pruners.BasePruner 객체

study_name: (기본값: None)

의미: 생성되는 Study의 이름을 지정합니다.
가능한 선택지: 문자열 또는 None

direction: (기본값: None)

의미: 최소화 또는 최대화로 설정할 목적 함수의 방향을 지정합니다.
가능한 선택지: 문자열 또는 StudyDirection 열거형 객체

load_if_exists: (기본값: False)

의미: 이미 같은 이름의 Study가 존재하는 경우, 해당 Study를 로드할지 여부를 결정합니다.
가능한 선택지: 불리언 (True 또는 False)

directions: (기본값: None)

의미: Study에 목적 함수가 여러 개인 경우, 각 목적 함수에 대한 방향을 지정합니다.
가능한 선택지: 문자열 또는 StudyDirection 열거형 객체의 리스트 또는 튜플

### 1. objective function
- 먼저 최적화 할 대상 함수인 objective 함수를 작성해야 합니다.
- 학습 로직 및 튜닝할 파라미터의 목록을 작성합니다.
  - Optuna에서 지원하는 Search Space 함수는 범주형, 실수형, 정수형, 이산 균등분포, 균등, 로그 균등 분포 등이 있습니다.
  - [Optuna search space 함수 리스트](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial)
- objective 함수의 반환 값은 우리가 튜닝하고자 하는 메트릭을 반환하면 됩니다.

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.0 MB/s eta 0:00:00


In [ ]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
def objective(trial):
    global x_train, x_valid, test
    # 튜닝할 파라미터 목록
    max_depth = trial.suggest_int('max_depth', 1, 10)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)

    # 데이터 전처리 함수로 전처리
    _x_train, _x_valid, x_test = preprocess(x_train, x_valid, test)

    # 모델 정의
    model = XGBRegressor(n_estimators=1000,
                        max_depth=max_depth,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        random_state=42)

    # 모델 학습
    model.fit(_x_train, y_train,
              eval_metric='mae',
              eval_set=[[_x_train, y_train], [_x_valid, y_valid]],
              early_stopping_rounds=30,
              verbose=None)

    # 검증 데이터 메트릭
    val_auc = mean_absolute_error(y_valid, model.predict(_x_valid))

    # 튜닝할 메트릭
    return val_auc

### 2. Study 정의 및 튜닝 시작하기
- objective 함수가 준비 되었다면, Optuna Study 객체를 생성 후 튜닝을 시작할 수 있습니다.
- study 객체 생성 시, 튜닝할 방향을 설정할 수 있습니다.
- study.optimize 메소드로 튜닝을 시작하며, objective 함수와 튜닝 횟수를 전달하면 됩니다.

In [ ]:
?optuna.create_study

- 옵션
optuna.create_study(*, storage: 'str | storages.BaseStorage | None' = None, sampler: "'samplers.BaseSampler' | None" = None, pruner: 'pruners.BasePruner | None' = None, study_name: 'str | None' = None, direction: 'str | StudyDirection | None' = None, load_if_exists: 'bool' = False, directions: 'Sequence[str | StudyDirection] | None' = None) -> 'Study'

In [ ]:
# study 생성 : 모델 학습

study = optuna.create_study(direction="maximize") # or minimize

[I 2023-11-15 02:32:24,551] A new study created in memory with name: no-name-98bac98d-1225-4b26-983f-321fb44e27ad


In [ ]:
study.optimize(objective, n_trials=10,)

[I 2023-11-15 02:32:25,818] Trial 0 finished with value: 1.4354744339787706 and parameters: {'max_depth': 10, 'subsample': 0.8045595510761383, 'colsample_bytree': 0.97805021803824}. Best is trial 0 with value: 1.4354744339787706.
[I 2023-11-15 02:32:26,567] Trial 1 finished with value: 1.4225270867047226 and parameters: {'max_depth': 5, 'subsample': 0.5165547977873107, 'colsample_bytree': 0.6647202924708415}. Best is trial 0 with value: 1.4354744339787706.
[I 2023-11-15 02:32:28,373] Trial 2 finished with value: 1.4211564555007603 and parameters: {'max_depth': 2, 'subsample': 0.7612570975327682, 'colsample_bytree': 0.9209940976034726}. Best is trial 0 with value: 1.4354744339787706.
[I 2023-11-15 02:32:30,670] Trial 3 finished with value: 1.4154794184497075 and parameters: {'max_depth': 2, 'subsample': 0.9068460340091611, 'colsample_bytree': 0.6939756714287784}. Best is trial 0 with value: 1.4354744339787706.
[I 2023-11-15 02:32:31,241] Trial 4 finished with value: 1.4239377793666481 a

### 3. Best Parameter
- 파라미터 튜닝이 끝났다면, study 객체에서 가장 좋은 성능을 냈던 파라미터와 메트릭을 확인할 수 있습니다.

In [ ]:
study.best_params, study.best_value

({'max_depth': 9,
  'subsample': 0.5139745426969449,
  'colsample_bytree': 0.761851642819963},
 1.457263344305215)

## Model Training
- HPO에서 선별한 파라미터로 모델을 학습합니다.

In [ ]:
best_params = study.best_params

In [ ]:
_x_train, _x_valid, x_test = preprocess(x_train, x_valid, test)

# 모델 정의
model = XGBRegressor(n_estimators=100,
                      max_depth=best_params['max_depth'],
                      subsample=best_params['subsample'],
                      colsample_bytree=best_params['colsample_bytree'],
                      random_state=42)

# 모델 학습
model.fit(_x_train, y_train,
          eval_metric='mae',
          eval_set=[[_x_train, y_train], [_x_valid, y_valid]],
          early_stopping_rounds=30,
          verbose=None)

# 검증 데이터 메트릭
val_mae = mean_absolute_error(y_valid, model.predict(_x_valid))

In [ ]:
val_mae

1.457263344305215

## Ensemble
개인적으로 앙상블은 머신러닝의 꽃이라고 생각합니다. 단일 모델로 좋은 성능을 이끄는 것도 중요하지만, 서로 다른 모델의 다양성을 고려하여 결과를 이끌어내는 앙상블은 응용할 수 있는 방법이 매우 많습니다.

### 1. Out-of-fold(OOF) Ensemble (같이 푸는 실습)
OOF 앙상블은 KFold 교차 검증에서 생성되는 각 Fold에 대한 예측 값을 앙상블하는 기법으로 모델 검증과 함께 앙상블을 진행할 수 있다는 장점이 있습니다.

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
val_scores = list()
oof_pred = np.zeros((test.shape[0], ))

skf = KFold(n_splits=5, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx], label[trn_idx]
    x_valid, y_valid = data.iloc[val_idx], label[val_idx]

    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)

    # 모델 정의
    model = XGBRegressor(n_estimators=100,
                         max_depth=best_params['max_depth'],
                         subsample=best_params['subsample'],
                         colsample_bytree=best_params['colsample_bytree'],
                         random_state=42,
                         tree_method='gpu_hist'
    )

    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 metric 확인
    trn_metric = mean_absolute_error(y_train, model.predict(x_train))
    val_metric = mean_absolute_error(y_valid, model.predict(x_valid))
    print('{} Fold, train metric : {:.4f}4, validation metric : {:.4f}'.format(i, trn_metric, val_metric))

    val_scores.append(val_metric)

    # 결과 모으기
    oof_pred += model.predict(x_test) / skf.n_splits

# 교차 검증 metric 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

0 Fold, train metric : 1.04134, validation metric : 1.5230
1 Fold, train metric : 1.03334, validation metric : 1.5193
2 Fold, train metric : 1.03084, validation metric : 1.5286
3 Fold, train metric : 1.02964, validation metric : 1.5446
4 Fold, train metric : 1.03204, validation metric : 1.5131
Cross Validation Score : 1.5257


### Submission

In [ ]:
submit = pd.read_csv(join(ROOT_PATH, 'data','sample_submission.csv'))

In [ ]:
submit['Age'] = oof_pred

In [ ]:
submit.to_csv("submission.csv", index=False)